In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
import os
os.chdir("/content/gdrive/My Drive/proje")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from random import sample
import pickle, cv2
#from scipy.misc import imresize
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import keras
from keras import applications, optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
import h5py
from keras.utils import plot_model



Using TensorFlow backend.


In [0]:
seed = 1337
np.random.seed(seed)

epochs = 5
batch_size = 25
input_shape = (224, 224, 3)
 
data = np.load('/content/gdrive/My Drive/proje/images_labels_224.npz')
x = data['x']
y = data['y']
n_classes = len(np.unique(y))    


In [0]:
def train_validation_split(x, y):
    # split data into training and test sets
    X_training, X_test, y_training, y_test = train_test_split(x, y, stratify=y, test_size= 0.1, random_state=1337)

    # split training into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_training, y_training, stratify=y_training, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test



In [0]:
X_train, X_val, X_test, y_train, y_val, y_test = train_validation_split(x, y)
print('Train data shape: ', X_train.shape) # 90% - 4752 (3564 + 1188)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape) # 10% - 528
print('Test labels shape: ', y_test.shape)

Train data shape:  (3564, 224, 224, 3)
Train labels shape:  (3564,)
Validation data shape:  (1188, 224, 224, 3)
Validation labels shape:  (1188,)
Test data shape:  (528, 224, 224, 3)
Test labels shape:  (528,)


In [0]:
def one_hot(y_train, y_val, y_test, n_classes):
    y_train = np_utils.to_categorical(y_train, n_classes)
    y_val = np_utils.to_categorical(y_val, n_classes)
    y_test = np_utils.to_categorical(y_test, n_classes)
    return y_train, y_val, y_test


In [0]:
y_train, y_val, y_test = one_hot(y_train, y_val, y_test, n_classes)

print(y_train)
print(y_val)
print(y_test)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [0]:
base_model = keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=None, input_shape=input_shape,  classes=10)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.25)(x)
predictions = Dense(n_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

model.summary()

W0708 12:17:06.290172 140605358884736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0708 12:17:06.334561 140605358884736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0708 12:17:06.353464 140605358884736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0708 12:17:06.398228 140605358884736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0708 12:17:07.118757 140605358884736 deprecation_wrapp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
# save the best model only
filepath ='/content/gdrive/My Drive/proje/model_VGG19.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, 
                                             monitor='val_acc', 
                                             verbose=1, 
                                             save_best_only=True, 
                                             mode='max')
callbacks_list = [checkpoint]


In [0]:
adam = Adam(lr=0.001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(loss=keras.losses.categorical_crossentropy,
              #optimizer=SGD(lr=0.1, momentum=0, decay=0, nesterov=False),
              #metrics=['accuracy'])

In [0]:
batch_size = 65
epochs = 10
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    verbose=1,
                    validation_data=(X_val, y_val))
    
    

Train on 3564 samples, validate on 1188 samples
Epoch 1/10
3564/3564 [==============================] - 67s 19ms/step - loss: 2.3151 - acc: 0.1080 - val_loss: 2.3027 - val_acc: 0.1002

Epoch 00001: val_acc improved from -inf to 0.10017, saving model to /content/gdrive/My Drive/proje/model_VGG19.hdf5
Epoch 2/10
3564/3564 [==============================] - 64s 18ms/step - loss: 2.3028 - acc: 0.0926 - val_loss: 2.3026 - val_acc: 0.1002

Epoch 00002: val_acc did not improve from 0.10017
Epoch 3/10
3564/3564 [==============================] - 63s 18ms/step - loss: 2.3027 - acc: 0.0999 - val_loss: 2.3026 - val_acc: 0.1002

Epoch 00003: val_acc did not improve from 0.10017
Epoch 4/10
3564/3564 [==============================] - 63s 18ms/step - loss: 2.3028 - acc: 0.0929 - val_loss: 2.3026 - val_acc: 0.1002

Epoch 00004: val_acc did not improve from 0.10017
Epoch 5/10
3564/3564 [==============================] - 63s 18ms/step - loss: 2.3028 - acc: 0.0937 - val_loss: 2.3026 - val_acc: 0.1002

E

In [0]:
history.history["acc"]


In [0]:
model.load_weights(filepath)
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.302692767345544
Test accuracy: 0.10037878787878787


In [0]:
plot_model(model, to_file='modelvgg19.png')

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [0]:
plt.subplot(2, 1, 1)
plt.title('Training loss')
plt.plot(history.history["loss"], 'o')
plt.xlabel('Iteration')

plt.subplot(2, 1, 2)
plt.title('Accuracy')
plt.plot(history.history["acc"], '-o', label='train')
plt.plot(history.history["val_acc"], '-o', label='val')
plt.plot([0.5] * len(history.history["val_acc"]), 'k--')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.gcf().set_size_inches(15, 12)
plt.show()